# Lab 2: Data Ingestion with Voyage-AI Embeddings into MongoDB Atlas

This lab focuses on preparing your data, generating vector embeddings using Voyage-AI, and storing these embeddings along with your text chunks in a MongoDB Atlas collection. This forms the indexing part of your RAG pipeline.

## Objectives
- Define sample text data (or load from a source).
- Use Voyage-AI to generate embeddings for text chunks.
- Store text chunks and their embeddings in MongoDB Atlas.
- Understand the structure of documents for vector search.

## Prerequisites
- Complete Lab 1: MongoDB Atlas Setup (ensuring `MONGO_URI` is set in `.env`).
- Obtain a Voyage-AI API Key and set it as `VOYAGE_API_KEY` in your `.env` file.
- Python environment set up with `pymongo`, `voyageai`, and `python-dotenv` installed.
  ```bash
  pip install pymongo voyageai python-dotenv
  ```

## Step 1: Load Environment Variables and Initialize Clients

In [ ]:
from dotenv import load_dotenv
import os
import voyageai
from pymongo import MongoClient

# Load environment variables
load_dotenv()

# Initialize Voyage-AI Client
voyage_api_key = os.environ.get("VOYAGE_API_KEY")
if not voyage_api_key:
    raise ValueError("VOYAGE_API_KEY not found in .env file or environment variables.")
vo = voyageai.Client(api_key=voyage_api_key)

# Initialize MongoDB Client
mongo_uri = os.environ.get("MONGO_URI")
if not mongo_uri:
    raise ValueError("MONGO_URI not found in .env file or environment variables.")
client = MongoClient(mongo_uri)

# Select your database and collection
# (These will be created if they don't exist upon first insertion)
db = client['rag_db']
collection = db['documents']

print("Clients initialized successfully.")

## Step 2: Prepare Sample Data

For this lab, we'll use a small array of text snippets. In a real-world scenario, you would typically load and chunk data from documents, articles, etc.

In [ ]:
sample_texts = [
    "The new product features include enhanced security protocols and faster processing.",
    "Our customer support is available 24/7 via live chat and email.",
    "This document outlines the privacy policy regarding user data collection and usage.",
    "Upcoming software updates will introduce a dark mode and custom themes.",
    "Please refer to the user manual for detailed installation instructions."
]

print(f"Prepared {len(sample_texts)} text chunks.")

## Step 3: Generate Embeddings with Voyage-AI

We'll use the `vo.embed()` method to convert our text chunks into vector embeddings. It's important to specify `input_type="document"` when embedding documents for your knowledge base.

In [ ]:
print("Generating embeddings with Voyage-AI...")
try:
    response = vo.embed(texts=sample_texts, model="voyage-large-2", input_type="document")
    embeddings = response.embeddings
    print(f"Generated {len(embeddings)} embeddings. Dimension: {len(embeddings[0])}")
except Exception as e:
    print(f"Error generating embeddings: {e}")
    # Exit or handle error appropriately
    exit()

## Step 4: Store Documents and Embeddings in MongoDB Atlas

Now, we'll create documents for MongoDB, each containing the original text chunk, its embedding, and some optional metadata (like `source`). Then we insert them into our collection.

In [ ]:
documents_to_insert = []
for i, text in enumerate(sample_texts):
    documents_to_insert.append({
        "text_chunk": text,
        "embedding": embeddings[i],
        "source": f"sample_doc_{i+1}" # Example metadata
    })

print(f"Preparing to insert {len(documents_to_insert)} documents...")

# Clear existing documents if you want a fresh start each time
# collection.delete_many({})

try:
    if documents_to_insert:
        # Check if collection is empty before inserting to avoid duplicates if run multiple times
        if collection.count_documents({}) == 0:
            insert_result = collection.insert_many(documents_to_insert)
            print(f"Successfully inserted {len(insert_result.inserted_ids)} documents into MongoDB.")
        else:
            print("Collection is not empty. Skipping insertion to avoid duplicates. Clear the collection manually if you want to re-insert.")
    else:
        print("No documents to insert.")
except Exception as e:
    print(f"Error inserting documents: {e}")

## Step 5: Verify Data and Plan Vector Search Index Creation

You can now go to your MongoDB Atlas UI, navigate to your cluster, and browse the `rag_db.documents` collection to see the inserted data.

To enable vector search, you need to create a Vector Search Index on the `embedding` field. This is typically done through the MongoDB Atlas UI.

1.  In MongoDB Atlas, go to your cluster.
2.  Click the **"Search"** tab.
3.  Click **"Create Search Index"**.
4.  Select **"JSON Editor"** as the configuration method.
5.  Copy and paste the following index definition:

    ```json
    {
      "mappings": {
        "dynamic": true,
        "fields": {
          "embedding": {
            "type": "knnVector",
            "numDimensions": 1536,  // **IMPORTANT:** Adjust to your model's embedding dimension (e.g., Voyage-AI large models are typically 1536)
            "similarity": "cosine"
          }
        }
      }
    }
    ```
    *Remember to adjust `numDimensions` if you use a different Voyage-AI model with a different embedding size.*
6.  Name the index `default` (or a name you prefer, but remember it for Lab 3).
7.  Click **"Create Index"**.

Wait for the index to build (it might take a few minutes). Once it's ready, you can proceed to Lab 3!

In [ ]:
# Don't forget to close the MongoDB client connection when done with your script/notebook
client.close()
print("MongoDB client connection closed.")